# Generate submission files directly from Google Sheet 

NOTE: Sensitive information was REMOVED for the public release of this repository.

We want to directly read from https://docs.google.com/spreadsheets/d/REMOVED.
- ID: REMOVED
- sheet: Sheet1 

- Setting up the oauth: [Google Spreadsheets and Python](https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html):
  - followed instructions for Google API
  - created project _SAMPL_ with Google Drive and Google Sheets APIs enabled
  - but gave account "scriptreadonly" only _Project / View_ privileges
  - downloaded as `oauth/service_client_readonly.json`
  - added _"scriptreadonly@REMOVED.iam.gserviceaccount.com"_ to Google Sheet with view privileges 
- Use code from [How to access Google Sheet data using the Python API and convert to Pandas dataframe](https://towardsdatascience.com/how-to-access-google-sheet-data-using-the-python-api-and-convert-to-pandas-dataframe-5ec020564f0e)

Install the Google API client
```bash
conda install google-api-python-client oauth2client
```

In [5]:
from apiclient.discovery import build
from httplib2 import Http
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

In [13]:
SPREADSHEET_ID = "REMOVED"
RANGE_NAME = "Sheet1"


def get_google_sheet(spreadsheet_id, range_name, secret="oauth/client_secret_readonly.json"):
    """ Retrieve sheet data using OAuth credentials and Google Python API. """
    scopes = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    # Setup the Sheets API
    scope = ['https://spreadsheets.google.com/feeds']
    creds = ServiceAccountCredentials.from_json_keyfile_name('oauth/client_secret_readonly.json', scope)

    service = build('sheets', 'v4', http=creds.authorize(Http()))

    # Call the Sheets API
    gsheet = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    return gsheet


def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df


gsheet = get_google_sheet(SPREADSHEET_ID, RANGE_NAME)
df = gsheet2df(gsheet)
print('Dataframe size = ', df.shape)
print(df.head())

Dataframe size =  (44, 16)
     ID forcefield DGwater errDGwater DGoct_dry errDGoct_dry DGoct_wet  \
0  SM02      AMBER  -33.58       0.49    -64.16         1.58    -64.66   
1  SM02  LigParGen  -37.63       1.21    -72.17         1.93    -73.60   
2  SM02     OPLSAA  -31.03       0.42    -63.68         1.66    -64.01   
3  SM02     CHARMM  -49.65       0.36                                    
4  SM04      AMBER  -37.34       0.39    -71.20         1.50    -71.66   

  errDGoct_wet  DDGwod errDDGwod logPwod errlogPwod  DDGwow errDDGwow logPwow  \
0         1.35   30.58      1.65    5.32       0.29   31.08      1.44    5.41   
1         2.32   34.54      2.28    6.01       0.40   35.97      2.62    6.26   
2         1.25   32.65      1.71    5.69       0.30   32.98      1.32    5.74   
3               -49.65      0.36   -8.64       0.06  -49.65      0.36   -8.64   
4         1.26   33.86      1.55    5.90       0.27   34.32      1.32    5.98   

  errlogPwow  
0       0.25  
1       0.4

In [14]:
df

,ID,forcefield,DGwater,errDGwater,DGoct_dry,errDGoct_dry,DGoct_wet,errDGoct_wet,DDGwod,errDDGwod,logPwod,errlogPwod,DDGwow,errDDGwow,logPwow,errlogPwow
0,SM02,AMBER,-33.58,0.49,-64.16,1.58,-64.66,1.35,30.58,1.65,5.32,0.29,31.08,1.44,5.41,0.25
1,SM02,LigParGen,-37.63,1.21,-72.17,1.93,-73.60,2.32,34.54,2.28,6.01,0.40,35.97,2.62,6.26,0.46
2,SM02,OPLSAA,-31.03,0.42,-63.68,1.66,-64.01,1.25,32.65,1.71,5.69,0.30,32.98,1.32,5.74,0.23
3,SM02,CHARMM,-49.65,0.36,,,,,-49.65,0.36,-8.64,0.06,-49.65,0.36,-8.64,0.06
4,SM04,AMBER,-37.34,0.39,-71.20,1.50,-71.66,1.26,33.86,1.55,5.90,0.27,34.32,1.32,5.98,0.23
5,SM04,LigParGen,-48.35,0.43,-81.97,1.93,-79.32,1.76,33.62,1.98,5.85,0.34,30.97,1.81,5.39,0.32
6,SM04,OPLSAA,-28.61,0.41,-66.92,1.24,-66.77,1.41,38.31,1.31,6.67,0.23,38.16,1.47,6.64,0.26
7,SM04,CHARMM,,,,,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,SM07,AMBER,-46.46,0.65,-74.03,2.27,-76.33,2.45,27.57,2.36,4.80,0.41,29.87,2.53,5.20,0.44
9,SM07,LigParGen,-56.42,0.39,-84.72,1.42,-82.41,1.69,28.30,1.47,4.93,0.26,25.99,1.73,4.53,0.30


In [18]:
df.to_csv("SAMPL6_gsheet.csv", index=False)